In [1]:
cd ../

/Users/arbaaz/Downloads/mabe/mabe_code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd


train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [4]:
from data_utils import get_col_names

cols = get_col_names("_sex")
cols

['mouse1_sex', 'mouse2_sex', 'mouse3_sex', 'mouse4_sex']

In [5]:
[train_metadata[col].isna().any() for col in cols]

[np.True_, np.True_, np.True_, np.True_]

In [6]:
[train_metadata[col].head() for col in cols]

[0    male
 1    male
 2    male
 3    male
 4    male
 Name: mouse1_sex, dtype: object,
 0    male
 1    male
 2    male
 3    male
 4    male
 Name: mouse2_sex, dtype: object,
 0    male
 1    male
 2    male
 3    male
 4    male
 Name: mouse3_sex, dtype: object,
 0    male
 1    male
 2    male
 3     NaN
 4     NaN
 Name: mouse4_sex, dtype: object]

In [7]:
# unique gender of mouse global
for col in cols:
    print(col)
    print(train_metadata[col].unique())
    print()

mouse1_sex
['male' 'female' nan]

mouse2_sex
['male' 'female' nan]

mouse3_sex
['male' nan]

mouse4_sex
['male' nan]



In [8]:
from collections import defaultdict


# unique gender of each mouse by lab

gender_by_lab = defaultdict(lambda: defaultdict(set))

for lab_id, grp in train_metadata.groupby("lab_id"):
    for col in cols:
        gender_by_lab[lab_id][col].update(grp[col].dropna().unique())
        
nested_dict = {
    lab: {k: ', '.join(v) if v else '' for k, v in inner.items()}
    for lab, inner in gender_by_lab.items()
}

# create dataframe
df = pd.DataFrame.from_dict(nested_dict, orient='index')

# display
df

,mouse1_sex,mouse2_sex,mouse3_sex,mouse4_sex
AdaptableSnail,male,male,male,male
BoisterousParrot,female,female,,
CRIM13,male,"male, female",,
CalMS21_supplemental,male,"male, female",,
CalMS21_task1,male,"male, female",,
CalMS21_task2,male,"male, female",,
CautiousGiraffe,"male, female","male, female",,
DeliriousFly,male,male,,
ElegantMink,male,"male, female",,
GroovyShrew,"male, female",female,,


In [9]:
import pandas as pd
from collections import defaultdict

# Step 1: collect counts
gender_counts = defaultdict(lambda: defaultdict(int))

for lab_id, grp in train_metadata.groupby("lab_id"):
    for col in cols:
        for strain, count in grp[col].value_counts().items():
            if pd.notna(strain):
                gender_counts[lab_id][strain] += count

# Step 2: convert to DataFrame
df = pd.DataFrame.from_dict(
    {lab: dict(strains) for lab, strains in gender_counts.items()},
    orient='index'
).fillna(0).astype(int)

df.index.name = 'lab_id'
df.columns.name = 'gender'

# Step 3: display
df

gender,male,female
lab_id,,
AdaptableSnail,61,0
CRIM13,30,12
CalMS21_supplemental,429,165
CalMS21_task1,141,61
CalMS21_task2,96,19
CautiousGiraffe,9,11
DeliriousFly,12,0
ElegantMink,23,15
GroovyShrew,11,23
